In [ ]:
import joblib 
import catboost
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

BEST_MODEL_PATH = "../../../baselines/catboost/models/best_catboost_transynergy_af_pandas.pkl"
transynergy_features_parquet_path: str | None = "../../../external/drug_combination/data/final_X.parquet"

X_df = pd.read_parquet(transynergy_features_parquet_path)
FEATURE_NAMES = X_df.columns

sns.set_theme(style="whitegrid")

## Load model

In [ ]:
model: catboost.CatBoostRegressor = joblib.load(BEST_MODEL_PATH)
model

## Load importances and interactions

In [ ]:
feature_importances = np.array(model.feature_importances_)
fi_df = pd.DataFrame({
    'feature': FEATURE_NAMES,
    'importance': feature_importances
})

fi_df = fi_df.sort_values(by='importance', ascending=False)
fi_df.shape

In [ ]:
fi_df.to_csv("catboost_feature_importances.csv")

In [ ]:
interactions = model.get_feature_importance(type='Interaction') 
# Format is [[1st_feature_idx, 2nd_feature_idx, importance], ...]

n_features = len(FEATURE_NAMES)
interaction_matrix = np.zeros((n_features, n_features))

# Fill matrix
for i_flt, j_flt, score in interactions:
    i = int(i_flt)
    j = int(j_flt)
    interaction_matrix[i][j] = score

interaction_matrix.shape

In [ ]:
interaction_df = pd.DataFrame(interaction_matrix, columns=FEATURE_NAMES)
interaction_df.index = FEATURE_NAMES
interaction_df.head()
interaction_df.to_csv("catboost_feature_interactions.csv")

In [ ]:
# TODO: add SHAP values, catboost library has method for it!

## Individual feature importances

In [ ]:
top_fi_df = fi_df.sort_values(by='importance', ascending=False).head(50)

plt.figure(figsize=(12, 10))  # Increase height for better readability
sns.barplot(
    data=top_fi_df,
    y='feature',
    x='importance',
    palette='viridis',
    hue='importance',
    legend=False
)
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Top 50 CatBoost Feature Importances")
plt.tight_layout()
plt.savefig('top_50_feature_importances.png')
plt.show()

## Feature interaction heatmap

In [ ]:
# Plot heatmap
## TOO MUCH DATA!
# plt.figure(figsize=(12, 10))
# sns.heatmap(interaction_matrix,
#             xticklabels=FEATURE_NAMES,
#             yticklabels=FEATURE_NAMES,
#             cmap='viridis',
#             square=True,
#             cbar_kws={'label': 'Interaction Importance'},
#             annot=False)

# plt.title('Feature Interaction Importance Heatmap')
# plt.tight_layout()
# plt.savefig('feature_interaction_heatmap.png')
# plt.show()

## Top 50 feature interactions

In [ ]:
# Step 1: Sort by interaction score descending
sorted_interactions = sorted(interactions, key=lambda x: x[2], reverse=True)

# Step 2: Take top 30
top_k = 30
top_interactions = sorted_interactions[:top_k]

# Step 3: Convert to (name1, name2, score) tuples
named_interactions = [
    (f"{FEATURE_NAMES[int(i)]} x {FEATURE_NAMES[int(j)]}", score)
    for i, j, score in top_interactions
]

# Step 4: Separate for plotting
labels, scores = zip(*named_interactions)

# Step 5: Plot
plt.figure(figsize=(20, 10))
sns.barplot(x=labels, y=scores, palette='viridis', hue=scores, legend=False)
plt.xticks(rotation=45, ha='right')
plt.xlabel("Interaction Score")
plt.ylabel("Feature Pair")
plt.title(f"Top {top_k} Feature Interactions by Importance")
plt.tight_layout()
plt.savefig('feature_interactions_top_30.png')
plt.show()

## Correlation analysis

In [ ]:
sns.heatmap(X_df[["SLCO1B1_CL", "SLCO1B1_A", "SLCO1B1_B", "PDCD2L_CL", "PDCD2L_A", "PDCD2L_B"]].corr(), annot=True)

In [ ]:
sns.heatmap(X_df[["BUB1B_CL", "BUB1B_A", "BUB1B_B", "EIF2S2_CL", "EIF2S2_A", "EIF2S2_B"]].corr(), annot=True)

In [ ]:
sns.heatmap(X_df[["MYB_CL", "MYB_A", "MYB_B", "FLCN_CL", "FLCN_A", "FLCN_B"]].corr(), annot=True)